<a href="https://colab.research.google.com/github/AceroMike/Big-Data/blob/main/Machine_Learning_with_Dask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will be using Dask to train several machine learning models. Dask is helpful because it allows us parallelize our code. First, our imports. 

In [1]:
#Installations
!pip install dask[complete] --quiet
!pip install dask distributed --upgrade --quiet
!pip install aiohttp --quiet
!pip install dask-ml --quiet

In [2]:
# Imports
import pandas as pd
import numpy as np 

#Dask 
import dask.bag as db
import dask.dataframe as dd
import dask.array as da
from dask import delayed
from dask.distributed import Client, progress
from dask_ml.model_selection import train_test_split
from dask_ml.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier #Joblib must be used if using sklearn model
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_validate, GridSearchCV

import joblib


#Warnings
import warnings
warnings.filterwarnings("ignore")


We will be using the [Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud) dataset from Kaggle to build 3 different classification models. We will be modeling using Logistic Regression, Random Forest Classifier, and Gradient Boosting Classifier. First, we want to load initialize a client and load the data the data set

In [3]:
client = Client(n_workers=4, threads_per_worker=2, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:37115 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 8.00 GB


In [4]:
# Load data into Dask Dataframe
df = dd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/creditcard.csv', dtype={'Time': 'float64'})
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


Now, we will be predicting the Class variable of our dataframe. The Class variable is equal to 1 in the case of fraud and 0 otherwise. WE will be using all of our features in the model. Then, we will train-test-split our data

In [5]:
# Defining X and y

# Features
X = df.drop("Class", axis=1)
# Target
Y = df["Class"]

#Train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [6]:
X_train

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
npartitions=3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


As we can see, X_train is not defined yet. It is a process waiting to happen. Since we can fit X_train, X_test, y_train, and y_test in memory. We will now persist.

In [7]:
# Persisting to RAM
X_train.persist()
X_test.persist()
y_train.persist()
y_test.persist()

Dask Series Structure:
npartitions=3
    int64
      ...
      ...
      ...
Name: Class, dtype: int64
Dask Name: split, 3 tasks

First, let's train our Logistic Regression model. We want to use Dask as the backend of the joblib library when applicable. however some models will not work with parallelization

**Logistic Regression**

In [10]:
from dask_ml.linear_model import LogisticRegression

In [19]:
lr = LogisticRegression()
lr.fit(X_train.values.compute(), y_train.values.compute())
    
preds_train = lr.predict(X_train.values.compute())
preds_test = lr.predict(X_test.values.compute())


In [20]:
# Print the scores
print("Logistic regression training score is: ", roc_auc_score(preds_train, y_train.values.compute()))
print("Logistic regression test score is: ", roc_auc_score(preds_test, y_test.values.compute()))

Logistic regression training score is:  0.9378350327726543
Logistic regression test score is:  0.9441022671439601


**Gradient Boosting**

In [14]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier #Joblib must be used if using sklearn model?

In [15]:
gbc = GradientBoostingClassifier()

with joblib.parallel_backend('dask'):
    gbc.fit(X_train.compute(), y_train.compute())
    
preds_train = gbc.predict(X_train.values.compute())
preds_test = gbc.predict(X_test.values.compute())

print("Gradient boosting tree training score is: ", roc_auc_score(preds_train, y_train.values.compute()))
print("Gradient boosting tree test score is: ", roc_auc_score(preds_test, y_test.values.compute()))

Gradient boosting tree training score is:  0.9539867700579169
Gradient boosting tree test score is:  0.9390692925570526


**Random Forest**

In [16]:
rfc = RandomForestClassifier()

with joblib.parallel_backend('dask'):
    rfc.fit(X_train.compute(), y_train.compute())
    
preds_train = rfc.predict(X_train.values.compute())
preds_test = rfc.predict(X_test.values.compute())

print("Random forest training score is: ", roc_auc_score(preds_train, y_train.values.compute()))
print("Random forest test score is: ", roc_auc_score(preds_test, y_test.values.compute()))

Random forest training score is:  0.9999956017082965
Random forest test score is:  0.9549175790421642


Comparing the models we see that the each model a high accuracy. However, the Random Forest Classifier worked best with the score of 0.955. We also see that all models except the Logistic Regression overfit the data. With the Random Forest model overfitting the most. 